In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
from os.path import join as pjoin
from tqdm.notebook import tqdm
import plotly.graph_objects as go
from scipy.stats import pearsonr, spearmanr, zscore
import itertools

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import circletrack_behavior as ctb
import circletrack_neural as ctn
import place_cells as pc
import plotting_functions as pf
import cell_overlap as co

/home/austinbaggetta/.conda/envs/calcium/lib/python3.10/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
## Settings
dpath = '../../../MultiCon_Imaging/MultiCon_Imaging2/output/'
fig_path = '../../../MultiCon_Imaging/MultiCon_Imaging2/intermediate_figures/'
bin_size = 0.4
velocity_thresh = 7
mouse_list = ['mc23', 'mc26']
session_list = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]

In [ ]:
## Cell overlap across days
## Distance of 4 pixels between cell centers
centroid_distance = 4
fig = pf.custom_graph_template(x_title='Day', y_title='', rows=1, columns=2, width=1000, 
                               shared_y=True, titles=mouse_list)
for idx, mouse in enumerate(mouse_list):
    mappings = pd.read_pickle(pjoin(dpath, f'cross_registration_results/circletrack_data/{mouse}/mappings_{centroid_distance}.pkl'))
    overlap = co.calculate_overlap(mappings)
    overlap = co.dates_to_days(overlap, '2023_10_21', days=20)
    matrix = overlap.pivot_table(index='session_id1', columns='session_id2', values='overlap')
    fig.add_trace(go.Heatmap(z=matrix.values, x=matrix.index, y=matrix.columns, coloraxis='coloraxis'), row=1, col=idx + 1)
boundaries = [5.5, 10.5, 15.5]
for boundary in boundaries:
        fig.add_vline(x=boundary, line_width=1.5, line_color='red', opacity=1)
        fig.add_hline(y=boundary, line_width=1.5, line_color='red', opacity=1) 
fig.update_yaxes(title='Day', col=1)
fig.update_layout(coloraxis_colorbar={'title': 'Percent Overlap'})
fig.show()
fig.write_image(pjoin(fig_path, 'mc23_mc26_overlap_heatmaps.png'))

In [ ]:
## Cell overlap across days
## Distance of 2.5 pixels between cell centers
centroid_distance = 2.5
fig = pf.custom_graph_template(x_title='Day', y_title='', rows=1, columns=2, width=900, 
                               shared_y=True, titles=mouse_list)
for idx, mouse in enumerate(mouse_list):
    mappings = pd.read_pickle(pjoin(dpath, f'cross_registration_results/circletrack_data/{mouse}/mappings_{centroid_distance}.pkl'))
    overlap = co.calculate_overlap(mappings)
    overlap = co.dates_to_days(overlap, '2023_10_21', days=20)
    matrix = overlap.pivot_table(index='session_id1', columns='session_id2', values='overlap')
    fig.add_trace(go.Heatmap(z=matrix.values, x=matrix.index, y=matrix.columns, coloraxis='coloraxis'), row=1, col=idx + 1)
boundaries = [5.5, 10.5, 15.5]
for boundary in boundaries:
        fig.add_vline(x=boundary, line_width=1.5, line_color='red', opacity=1)
        fig.add_hline(y=boundary, line_width=1.5, line_color='red', opacity=1) 
fig.update_yaxes(title='Day', col=1)
# fig.update_layout(coloraxis_colorbar={'title': 'Percent Overlap'})
fig.show()

In [ ]:
## Plot of average cell overlap comparing all contexts
day_tup_list = [((0, 5), (0, 5)), ((0, 5), (6, 10)), ((0, 5), (11, 15)), ((0, 5), (16, 20)),
                ((6, 10), (0, 5)), ((6, 10), (6, 10)), ((6, 10), (11, 15)), ((6, 10), (16, 20)),
                ((11, 15), (0, 5)), ((11, 15), (6, 10)), ((11, 15), (11, 15)), ((11, 15), (16, 20)),
                ((16, 20), (0, 5)), ((16, 20), (6, 10)), ((16, 20), (11, 15)), ((16, 20), (16, 20))]
output_df = pd.DataFrame()
last_index = 0
for mouse in mouse_list:
    mappings = pd.read_pickle(pjoin(dpath, f'cross_registration_results/circletrack_data/{mouse}/mappings_{centroid_distance}.pkl'))
    overlap = co.calculate_overlap(mappings)
    overlap = co.dates_to_days(overlap, '2023_10_21', days=20)
    matrix = overlap.pivot_table(index='session_id1', columns='session_id2', values='overlap')
    for idx, tup in enumerate(day_tup_list):
        if idx <= 3:
            context = 'A'
        elif (idx > 3) & (idx <= 7):
            context = 'B'
        elif (idx > 7) & (idx <= 11):
            context = 'C'
        elif idx > 11:
            context = 'D'
        df = co.average_overlap_across_contexts(matrix, start_tuple=tup[0], end_tuple=tup[1])
        output_df = pd.concat([output_df, df], ignore_index=True)
        output_df.loc[idx+last_index, 'base_context'] = context
        output_df.loc[idx+last_index, 'mouse'] = mouse
    last_index = last_index + idx + 1
output_df['compared_to'] = ['A', 'B', 'C', 'D'] * (len(mouse_list)*4)
average_overlap = output_df.groupby(['compared_to', 'base_context'], as_index=False).agg({'avg': ['mean', 'sem']})

In [ ]:
## Plot of average cell overlap comparing all contexts for all mice
fig = pf.custom_graph_template(x_title='', y_title='Average Percent Overlap (%)')
for context in np.unique(average_overlap['compared_to']):
    plot_data = average_overlap[average_overlap['compared_to'] == context]
    fig.add_trace(go.Scatter(x=plot_data['base_context'], y=plot_data['avg']['mean'], mode='lines+markers', name=context,
                             error_y=dict(type='data', array=plot_data['avg']['sem'])))
fig.show()

In [ ]:
## Bar plot of cell overlap at each context switch
centroid_distance = 2.5
last_days_in_context = [5, 10, 15]
fig = pf.custom_graph_template(x_title='', y_title='Percent Overlap (%)')
data = {'mouse': [], 'A to B': [], 'B to C': [], 'C to D': []}
for mouse in tqdm(mouse_list):
    if mouse == 'mc23':
        pass
    else:
        mappings = pd.read_pickle(pjoin(dpath, f'cross_registration_results/circletrack_data/{mouse}/mappings_{centroid_distance}.pkl'))
        overlap = co.calculate_overlap(mappings)
        overlap = co.dates_to_days(overlap, '2023_10_21', days=20)
        data['mouse'].append(mouse)
        data['A to B'].append(overlap['overlap'][(overlap['session_id1'] == last_days_in_context[0]) & (overlap['session_id2'] == last_days_in_context[0]+1)].values[0])
        data['B to C'].append(overlap['overlap'][(overlap['session_id1'] == last_days_in_context[1]) & (overlap['session_id2'] == last_days_in_context[1]+1)].values[0])
        data['C to D'].append(overlap['overlap'][(overlap['session_id1'] == last_days_in_context[2]) & (overlap['session_id2'] == last_days_in_context[2]+1)].values[0])
df = pd.DataFrame(data)
df = df.melt(id_vars='mouse', var_name='switch', value_name='overlap')
fig.add_trace(go.Bar(x=df['switch'], y=df['overlap'], marker_line_color='black',
                     marker_line_width=2, opacity=0.8))
fig.show()

In [ ]:
context_days = {'A': [1, 2, 3, 4, 5], 'B': [6, 7, 8, 9, 10], 'C': [11, 12, 13, 14, 15], 'D': [16, 17, 18, 19, 20]}
overlap[(overlap['session_id1'] == 1) & (overlap['session_id2'].isin(context_days['B']))]

In [ ]:
context_days = {'A': [1, 2, 3, 4, 5], 'B': [6, 7, 8, 9, 10], 'C': [11, 12, 13, 14, 15], 'D': [16, 17, 18, 19, 20]}
fig = pf.custom_graph_template(x_title='Day', y_title='Cell Overlap (%)', titles=['B compared to A'])
for day in context_days['A']:
    plot_data = overlap[(overlap['session_id1'] == day) & (overlap['session_id2'].isin(context_days['B']))]
    fig.add_trace(go.Scatter(x=plot_data['session_id2'], y=plot_data['overlap'], mode='markers', name=f'Day {day}',
                             marker=dict(line=dict(width=1)), opacity=0.7))
fig.show()

In [ ]:
fig = pf.custom_graph_template(x_title='Day', y_title='Cell Overlap (%)', titles=['C compared to B'])
for day in context_days['B']:
    plot_data = overlap[(overlap['session_id1'] == day) & (overlap['session_id2'].isin(context_days['C']))]
    fig.add_trace(go.Scatter(x=plot_data['session_id2'], y=plot_data['overlap'], mode='markers', name=f'Day {day}',
                             marker=dict(line=dict(width=1)), opacity=0.7))
fig.show()

In [ ]:
fig = pf.custom_graph_template(x_title='Day', y_title='Cell Overlap (%)', titles=['D compared to C'])
for day in context_days['C']:
    plot_data = overlap[(overlap['session_id1'] == day) & (overlap['session_id2'].isin(context_days['D']))]
    fig.add_trace(go.Scatter(x=plot_data['session_id2'], y=plot_data['overlap'], mode='markers', name=f'Day {day}',
                             marker=dict(line=dict(width=1)), opacity=0.7))
fig.show()

In [ ]:
fig = pf.custom_graph_template(x_title='Day', y_title='', rows=1, columns=3, shared_y=True,
                               titles=['B compared to A', 'C compared to B', 'D compared to C'], width=1000)
for day in context_days['A']:
    plot_data = overlap[(overlap['session_id1'] == day) & (overlap['session_id2'].isin(context_days['B']))]
    fig.add_trace(go.Scatter(x=plot_data['session_id2'], y=plot_data['overlap'], mode='markers', name=f'Day {day}',
                             marker=dict(line=dict(width=1)), opacity=0.7), row=1, col=1)
    
for day in context_days['B']:
    plot_data = overlap[(overlap['session_id1'] == day) & (overlap['session_id2'].isin(context_days['C']))]
    fig.add_trace(go.Scatter(x=plot_data['session_id2'], y=plot_data['overlap'], mode='markers', name=f'Day {day}',
                             marker=dict(line=dict(width=1)), opacity=0.7), row=1, col=2)
    
for day in context_days['C']:
    plot_data = overlap[(overlap['session_id1'] == day) & (overlap['session_id2'].isin(context_days['D']))]
    fig.add_trace(go.Scatter(x=plot_data['session_id2'], y=plot_data['overlap'], mode='markers', name=f'Day {day}',
                             marker=dict(line=dict(width=1)), opacity=0.7), row=1, col=3)
fig.update_yaxes(title='Cell Overlap (%)', col=1)
fig.show()

### Calculate place metrics for overlapping and nonoverlapping cells between all sessions.

In [9]:
centroid_distance = 4
data_type = 'S'
alpha = 0.001
nbins = 20
for idx, mouse in enumerate(['mc26']):
    overlap_results = {'mouse': [], 'sess1': [], 'sess2': [], 'place_cells_overlap_sess2': [], 'total_cells_overlap_sess2': [], 
                       'place_cells_nonoverlap_sess2': [], 'total_cells_nonoverlap_sess2': [], 'overlap_ids_sess2': [], 'nonoverlap_ids_sess2': [],
                       'overlap_place_ids_sess2': [], 'nonoverlap_place_ids_sess2': []}
    mappings = pd.read_pickle(pjoin(dpath, f'cross_registration_results/circletrack_data/{mouse}/mappings_{centroid_distance}.pkl'))
    mpath = pjoin(dpath, f'aligned_minian/{mouse}/{data_type}')
    for first_loop_sess in tqdm(os.listdir(mpath)):
        sess_one = xr.open_dataset(pjoin(mpath, first_loop_sess))[data_type]
        T_one = ctn.calculate_otsu_thresh(sess_one.values)

        for second_loop_sess in os.listdir(mpath):
            sess_two = xr.open_dataset(pjoin(mpath, second_loop_sess))[data_type]
            T_two = ctn.calculate_otsu_thresh(sess_two.values)
            cell_pairs = mappings.session[[sess_one.attrs['date'], sess_two.attrs['date']]].dropna(how='any').drop_duplicates().reset_index(drop=True)

            ## Select overlapping units and non-overlapping units
            overlap_sess_one = sess_one.sel(unit_id=cell_pairs.iloc[:, 0].values)
            nonoverlap_sess_one = sess_one.sel(unit_id=np.array([i for i in sess_one['unit_id'].values if i not in cell_pairs.iloc[:, 0].values]))
            overlap_sess_two = sess_two.sel(unit_id=cell_pairs.iloc[:, 1].values)
            nonoverlap_sess_two = sess_two.sel(unit_id=np.array([i for i in sess_two['unit_id'].values if i not in cell_pairs.iloc[:, 1].values]))

            pf_overlap_two = pc.PlaceFields(x=ctb.convert_to_cm(x=overlap_sess_two['x'], y=overlap_sess_two['y'], update_dataframe=False)[0],
                                y=ctb.convert_to_cm(x=overlap_sess_two['x'], y=overlap_sess_two['y'], update_dataframe=False)[1],
                                t=overlap_sess_two['behav_t'].values,
                                neural_data=(overlap_sess_two.values > T_two).astype(int),
                                circular=False, shuffle_test=True, nbins=nbins)
            overlap_sess_two = overlap_sess_two.assign_coords(place_cell_r = ('unit_id', pf_overlap_two.data['spatial_info_pvals'] < alpha))

            pf_nonoverlap_two = pc.PlaceFields(x=ctb.convert_to_cm(x=nonoverlap_sess_two['x'], y=nonoverlap_sess_two['y'], update_dataframe=False)[0],
                                            y=ctb.convert_to_cm(x=nonoverlap_sess_two['x'], y=nonoverlap_sess_two['y'], update_dataframe=False)[1],
                                            t=nonoverlap_sess_two['behav_t'].values,
                                            neural_data=(nonoverlap_sess_two.values > T_two).astype(int),
                                            circular=False, shuffle_test=True, nbins=nbins)
            nonoverlap_sess_two = nonoverlap_sess_two.assign_coords(place_cell_r = ('unit_id', pf_nonoverlap_two.data['spatial_info_pvals'] < alpha))

            overlap_results['mouse'].append(mouse)
            overlap_results['sess1'].append(sess_one.attrs['date'])
            overlap_results['sess2'].append(sess_two.attrs['date'])
            overlap_results['place_cells_overlap_sess2'].append(np.sum(overlap_sess_two['place_cell_r'].values))
            overlap_results['total_cells_overlap_sess2'].append(overlap_sess_two.shape[0])
            overlap_results['place_cells_nonoverlap_sess2'].append(np.sum(nonoverlap_sess_two['place_cell_r'].values))
            overlap_results['total_cells_nonoverlap_sess2'].append(nonoverlap_sess_two.shape[0])
            overlap_results['overlap_ids_sess2'].append(overlap_sess_two['unit_id'].values)
            overlap_results['nonoverlap_ids_sess2'].append(nonoverlap_sess_two['unit_id'].values)
            overlap_results['overlap_place_ids_sess2'].append(overlap_sess_two['unit_id'].values[overlap_sess_two['place_cell_r']])
            overlap_results['nonoverlap_place_ids_sess2'].append(nonoverlap_sess_two['unit_id'].values[nonoverlap_sess_two['place_cell_r']])
    results_df = pd.DataFrame(overlap_results)
    save_path = pjoin(dpath, f'overlapping_cells/{mouse}')
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    results_df.to_feather(pjoin(save_path, 'overlapping_cells.feat'))

  0%|          | 0/18 [00:00<?, ?it/s]

/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:44: RuntimeWarning: invalid value encountered in divide
  velocity = dists / np.diff(t, prepend = 0) ## in seconds
/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:392: RuntimeWarning: invalid value encountered in divide
  pf = pf / self.data["occupancy_map"]


Doing shuffle tests. This may take a while.


/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:458: RuntimeWarning: invalid value encountered in scalar divide
  SI_z = (self.data["spatial_information"][neuron] - np.mean(shuffled_SIs)) / np.std(
/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:44: RuntimeWarning: invalid value encountered in divide
  velocity = dists / np.diff(t, prepend = 0) ## in seconds
/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:392: RuntimeWarning: invalid value encountered in divide
  pf = pf / self.data["occupancy_map"]


Doing shuffle tests. This may take a while.


...

Doing shuffle tests. This may take a while.


/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:44: RuntimeWarning: invalid value encountered in divide
  velocity = dists / np.diff(t, prepend = 0) ## in seconds
/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:392: RuntimeWarning: invalid value encountered in divide
  pf = pf / self.data["occupancy_map"]


Doing shuffle tests. This may take a while.


/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:44: RuntimeWarning: invalid value encountered in divide
  velocity = dists / np.diff(t, prepend = 0) ## in seconds
/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis/place_cells.py:392: RuntimeWarning: invalid value encountered in divide
  pf = pf / self.data["occupancy_map"]


Doing shuffle tests. This may take a while.


In [10]:
results_df

,mouse,sess1,sess2,place_cells_overlap_sess2,total_cells_overlap_sess2,place_cells_nonoverlap_sess2,total_cells_nonoverlap_sess2,overlap_ids_sess2,nonoverlap_ids_sess2,overlap_place_ids_sess2,nonoverlap_place_ids_sess2
0,mc26,2023_10_21,2023_10_21,28,94,2,7,"[25, 52, 78, 0, 44, 88, 81, 21, 51, 36, 6, 80,...","[14, 26, 27, 47, 54, 63, 69]","[78, 21, 51, 95, 46, 68, 13, 90, 91, 56, 100, ...","[14, 47]"
1,mc26,2023_10_21,2023_10_22,21,69,5,31,"[25, 51, 72, 0, 42, 81, 77, 22, 49, 33, 10, 75...","[1, 2, 4, 5, 11, 12, 16, 26, 27, 29, 30, 31, 3...","[81, 22, 75, 48, 19, 74, 86, 88, 6, 66, 7, 43,...","[29, 30, 71, 87, 95]"
2,mc26,2023_10_21,2023_10_23,11,74,11,71,"[77, 111, 76, 82, 25, 89, 11, 108, 74, 99, 114...","[0, 2, 3, 4, 5, 8, 9, 12, 13, 14, 16, 18, 20, ...","[25, 107, 35, 51, 102, 67, 127, 81, 6, 118, 135]","[12, 18, 26, 27, 36, 38, 41, 45, 61, 93, 128]"
3,mc26,2023_10_21,2023_10_24,27,51,24,59,"[68, 45, 30, 90, 64, 29, 56, 101, 72, 48, 32, ...","[0, 1, 2, 3, 4, 6, 8, 9, 11, 12, 15, 16, 17, 1...","[45, 30, 64, 29, 101, 69, 65, 76, 109, 96, 87,...","[0, 1, 2, 3, 6, 8, 9, 12, 15, 16, 19, 24, 26, ..."
4,mc26,2023_10_21,2023_10_25,28,58,36,97,"[130, 115, 81, 8, 110, 101, 31, 121, 140, 78, ...","[0, 1, 2, 3, 4, 6, 7, 10, 12, 13, 16, 17, 18, ...","[115, 8, 110, 31, 121, 112, 28, 56, 14, 55, 15...","[7, 10, 27, 29, 37, 42, 48, 49, 50, 57, 63, 64..."
...,...,...,...,...,...,...,...,...,...,...,...
319,mc26,2023_11_09,2023_11_04,45,98,49,99,"[112, 15, 21, 25, 78, 89, 6, 71, 85, 38, 95, 6...","[0, 2, 3, 4, 5, 7, 8, 9, 11, 12, 16, 23, 26, 2...","[15, 25, 78, 6, 85, 63, 154, 145, 155, 56, 10,...","[0, 2, 3, 4, 5, 7, 12, 23, 28, 32, 39, 40, 46,..."
320,mc26,2023_11_09,2023_11_05,26,114,29,126,"[131, 179, 134, 138, 190, 49, 56, 204, 16, 229...","[0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 14, 15, 17, 19...","[190, 49, 18, 29, 160, 30, 132, 159, 104, 53, ...","[2, 4, 8, 21, 48, 51, 72, 76, 77, 89, 98, 115,..."
321,mc26,2023_11_09,2023_11_06,35,110,25,78,"[98, 15, 72, 50, 10, 66, 0, 4, 75, 18, 22, 119...","[2, 5, 7, 9, 12, 13, 14, 17, 19, 23, 30, 31, 3...","[15, 72, 22, 155, 126, 113, 79, 82, 16, 24, 1,...","[2, 7, 9, 41, 44, 52, 54, 57, 70, 80, 94, 117,..."
322,mc26,2023_11_09,2023_11_07,24,124,12,67,"[112, 14, 35, 36, 28, 166, 71, 81, 92, 5, 63, ...","[1, 4, 6, 10, 13, 17, 18, 19, 21, 27, 29, 40, ...","[160, 7, 120, 91, 78, 8, 141, 125, 187, 139, 1...","[1, 19, 40, 52, 62, 65, 102, 114, 136, 162, 17..."


### The code below is to calculate how many centimeters one pixel represents on the circular track.

In [ ]:
import re
import sys
import os
from os.path import join as pjoin

import numpy as np
import pandas as pd
from natsort import natsort_keygen
from tqdm import tqdm

sys.path.append('/home/austinbaggetta/csstorage3/CircleTrack/CircleTrackAnalysis')
import circletrack_behavior as ctb

# %%
## Set parameters
parent_dir = 'Archive'
experiment_dir = 'Measurement_Video'
behavior_path = os.path.abspath(f"../../../{parent_dir}/{experiment_dir}/circletrack_data/")
output_path = os.path.abspath(f"../../../{parent_dir}/{experiment_dir}/output/behav")
if not os.path.exists(output_path):
    os.makedirs(output_path)
mouse_list = ['mca_test']
## Set str2match variable (regex for mouse name)
str2match = "(mca_test)"
downsample = False
if downsample:
    sampling_rate = 1/30 ## started keeping minian output at 1/30
    frame_count = 20 * 60 / sampling_rate ## session length in minutes x 60s per minute / sampling rate
## Set relative path variable for circletrack behavior data
csv_path = pjoin(behavior_path, "data/**/**/**/circle_track.csv")
log_path = pjoin(behavior_path, "data/**/**/**/**.log")
## Create list of files
file_list = ctb.get_file_list(csv_path)
log_list = ctb.get_file_list(log_path)
## Loop through file_list to extract mouse name
mouseID = []
for file in file_list:
    mouse = ctb.get_mouse(file, str2match)
    mouseID.append(mouse)
## Combine file_list and mouseID
combined_list = ctb.combine(file_list, mouseID)
combined_log = ctb.combine(log_list, mouseID)

for mouse in mouse_list:
    print(mouse)
    natsort_key = natsort_keygen() 
    subset = ctb.subset_combined(combined_list, mouse).reset_index(drop=True)
    subset_log = ctb.subset_combined(combined_log, mouse).reset_index(drop=True)
    subset = sorted(subset, key=natsort_key)
    subset_log = sorted(subset_log, key=natsort_key)
    circle_track = pd.read_csv(subset[0])
    reward_one, reward_two = ctb.get_rewarding_ports(circle_track, processed=False)
    rewards = [reward_one, reward_two]
    for idx in np.arange(0, len(circle_track['event'])):
        if 'probe' in circle_track['event'][idx]:
            probe_end = float(re.search('probe length: ([0-9]+)', circle_track['event'][idx])[1])
        else:
            pass
    circle_track = ctb.crop_data(circle_track)
    unix_start =  pd.to_numeric(circle_track.loc[circle_track['event'] == 'START', 'timestamp'].values[0])
    circle_track["frame"] = np.arange(len(circle_track))
    data_out = circle_track[circle_track["event"] == "LOCATION"].copy().reset_index(drop=True)
    data_out['timestamp'] = pd.to_numeric(data_out['timestamp']) ## sometimes saved as string, not sure why
    if downsample:
        time_vector = np.arange(unix_start, (frame_count * sampling_rate + unix_start), sampling_rate)
        arg_mins = [np.abs(data_out['timestamp'] - t).argmin() for t in time_vector] ## resample to sampling freq of time_vector
        data_out = data_out.loc[arg_mins, :].reset_index(drop=True)
    events = circle_track[circle_track["event"] != "LOCATION"].copy().reset_index(drop=True)
    events['timestamp'] = pd.to_numeric(events['timestamp'])
    data_out['t'] = (data_out['timestamp'] - unix_start)
    data_out[["x", "y", "a_pos"]] = (
        data_out["data"]
        .apply(
            lambda d: pd.Series(
                re.search(
                    r"X(?P<x>[0-9]+)Y(?P<y>[0-9]+)A(?P<ang>[0-9]+)", d
                ).groupdict()
            )
        )
        .astype(float)
    )
    data_out["lick_port"] = -1
    data_out["water"] = False
    for _, row in events.iterrows():
        ts = row["timestamp"]
        idx = np.argmin(np.abs(data_out["timestamp"] - ts))
        try:
            port = int(row["data"][-1])
        except TypeError:
            continue
        data_out.loc[idx, "lick_port"] = port
        if row["event"] == "REWARD":
            data_out.loc[idx, "water"] = True
    data_out['animal'] = mouse
    data_out["trials"] = ctb.get_trials(
        data_out, shift_factor=0, angle_type="radians", counterclockwise=True
    )
    data_out["lin_position"] = ctb.linearize_trajectory(
        data_out, angle_type="radians", shift_factor=0
    )
    data_out[['reward_one', 'reward_two']] = int(rewards[0][-1]), int(rewards[1][-1])
    data_out = (
        data_out.drop(columns=["event", "data"])
        .rename(columns={"timestamp": "unix"})
        .reset_index(drop=True)
    )
    data_out['probe'] = data_out['t'] < probe_end
    if pd.isna(subset_log[0]):
        data_out['maze'] = 'No behavior log'
    else:
        data_out['maze'] = subset_log[0][-9:-4]
    result_path = pjoin(output_path, mouse)
    if not os.path.exists(result_path):
        os.makedirs(result_path)
        data_out.to_feather(pjoin(result_path, "{}_{}.feat".format(mouse, '1'))) ## label with day number

In [ ]:
## Maze is 30 inches from outer wall to outer wall. 30 inches is 76.2 centimeters.
## Dividing the average pixel distance of the x and y distances by the total length in centimeters
## should give the amount of pixels per centimeter.
xmax, xmin = np.max(data_out['x']), np.min(data_out['x'])
ymax, ymin = np.max(data_out['y']), np.min(data_out['y'])
avg_dist = ((xmax - xmin) + (ymax - ymin)) / 2
avg_dist / 76.2 ## cm

In [ ]:
data_out = ctb.convert_to_cm(data_out)
data_out